Analytic Inverse kinematic solver for simplified model of leg

In [2]:
import pybullet
import time
import pybullet_data
import numpy as np

In [27]:
phisycsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())

Spawn the Robot...

In [115]:
pybullet.resetSimulation()
planeID = pybullet.loadURDF("plane.urdf")
pybullet.setGravity(0,0,-9.81)
robotID = pybullet.loadURDF("simple_robot/urdf/simple_robot.urdf",
                         [0.0,0.0,0.8],pybullet.getQuaternionFromEuler([0.0,0.0,-np.pi/2]),useFixedBase = 1)
pybullet.setRealTimeSimulation(1)

Check Joints + Home

In [108]:
for index in range(pybullet.getNumJoints(robotID)):
    print(pybullet.getJointInfo(robotID,index)[0:2],
          pybullet.getJointState(robotID,pybullet.getJointInfo(robotID,index)[0])[0])

homeConfig = list([0,0,-20*np.pi/180,40*np.pi/180,-20*np.pi/180,0])
for index in range(pybullet.getNumJoints(robotID)):
    pybullet.setJointMotorControl2(bodyIndex=robotID,
                            jointIndex=index,
                            controlMode=pybullet.POSITION_CONTROL,
                            targetPosition = homeConfig[index])

(0, b'base_hip_yaw_joint') 7.855645070348279e-05
(1, b'hip_yaw_roll_joint') 0.005903000906134951
(2, b'hip_roll_pitch_joint') -0.3607959571203137
(3, b'hip_knee_joint') 1.4602315725390413
(4, b'knee_ankle_pitch_joint') 0.18847356271783167
(5, b'ankle_pitch_roll_joint') 0.04118704290322368


Inverse Kinematic Solver (Bullet Built-in)

In [94]:
## create simple trajectory for ankle
trajectory = list("")
for i in range(300):
    trajectory.append(np.array([0.0,-0.1+0.001*i,0.15]))
## Solve IK and set Joint angles
for i in range(300):
    
    config = pybullet.calculateInverseKinematics(robotID,5,
                                                 trajectory[i],pybullet.getQuaternionFromEuler([0,0,np.pi/2])) 
    """config = calculateIK(trajectory[i],[0,0,np.pi/2])"""
    for index in range(pybullet.getNumJoints(robotID)):
        pybullet.setJointMotorControl2(bodyIndex=robotID,
                                jointIndex=index,
                                controlMode=pybullet.POSITION_CONTROL,
                                targetPosition = config[index])
    time.sleep(0.02)

Inverse Malek

In [ ]:
L1 = 0.135
L2 = 0.155

def malek_IK(X,Y,Z,theta1):
    theta4 = np.arccos(X**2 + Y**2  + Z**2 - L1**2 - L2**2)/(2*L1*L2)
    theta2 = np.arctan(Y/(-X * np.sin(theta1) - Z * np.cos(theta1)))
    
    k1 = L1 + L2 * np.cos(theta4)
    k2 = L2 * np.sin(theta4)
    A = k1 * np.sin(theta3) - k2 * np.sin(theta3)
    B = k1 * np.sin(theta3) + k2 * np.sin(theta3)
    
    theta5 = np.arctan(-np.tan(theta5)np.cos(theta5))

Geometric (Kajita)

In [174]:
def Rroll(phi):
    R = np.eye(3)
    R[1,1] = np.cos(phi)
    R[1,2] = -np.sin(phi)
    R[2,1] = np.sin(phi)
    R[2,2] = np.cos(phi)
    return R

def Rpitch(theta):
    R = np.eye(3)
    R[0,0] = np.cos(theta)
    R[0,2] = np.sin(theta)
    R[2,0] = -np.sin(theta)
    R[2,2] = np.cos(theta)
    return R

print(Rpitch(np.pi/2))

[[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]


In [186]:
def geometric_ik(p1,R1,p7,R7, A , d, B):
    """
        inputs:
            a -----------> Length of Hip
            b -----------> length of Shank
            (p1, R1) ----> position and attitude of body
            (p7, R7) ----> position and attitude of right foot
        output
            joint angles --> q = [q2,q3,q4,q5,q6,q7]
    """
    D = np.array([0,d,0])
    #E = np.array([0,0,e])
    r = np.matmul(R7.T , (p1 + np.matmul(R1 , D) - p7))
    C = np.sqrt(r[0]**2 + r[1]**2 +r[2]**2)
    c5 = (C**2 - A**2 - B**2) / (2*A*B)
    if c5 >= 1:
        q5 = 0.0
    elif c5 <= -1:
        q5 = np.pi
    else:
        q5 = np.arccos(c5) # Knee Pitch    
    q6a = np.arcsin((A/C)*np.sin(np.pi-q5)) # Ankle Pitch Sub
    q7 = np.arctan2(r[1],r[2]) # Ankle Roll
    if q7 > np.pi/2:
        q7 -= np.pi
    elif q7 < -np.pi/2:
        q7 += np.pi
    q6 = -np.arctan2(r[0], np.sign(r[2]) * np.sqrt(r[1]**2 + r[2]**2)) - q6a # Ankle Pitch
    
    R = np.matmul(R1.T , np.matmul(R7 , np.matmul(Rroll(-q7) , Rpitch(-q6-q5))))
    q2 = np.arctan2(-R[0,1], R[1,1]) # Hip Yaw
    q3 = np.arctan2(R[2,1], -R[0,1] * np.sin(q2) + R[1,1] * np.cos(q2))  # Hip Roll
    q4 = np.arctan2(-R[2,0], R[2,2]) # Hip Pitch
                     
    return([q2,q3,q4,q5,q6,q7])

print(pybullet.calculateInverseKinematics(robotID,5,
                                        [0.0,0.1,0.45],pybullet.getQuaternionFromEuler([0,0,np.pi/2])))

print("-------------------------------------------")
r1 = np.eye(3)
r2 = np.array([[0,0,1],
              [0,1,0],
              [-1,0,0]])
print(geometric_ik([0.0,0.0,0.6],r1,[0,0.1,0.45],r2,0.155,0,0.135))

(1.5703675031282334, 0.03162130279679448, -0.9241153113708983, 1.5972585219175928, -0.6790858713194319, -0.03214143750793551)
-------------------------------------------
[-1.5707963267948966, 0.0, -0.8157489848799928, 1.80593198240334, 0.5806133292715496, -1.5707963267948966]


Test Geometric Solution

In [197]:
## Create Same trajectory
trajectory = list("")
for i in range(200):
    trajectory.append(np.array([-0.2 + 0.002*i,0.0,0.4]))
    
# define rotation matrix of foot attitude and base attitude
r2 = np.eye(3)
r1 = np.eye(3)
## Solve IK and set Joint angles
for i in range(200):
    
    config = geometric_ik([0.0,0.0,0.8-0.21],r1,trajectory[i],r2,0.155,0,0.135)
    for index in range(pybullet.getNumJoints(robotID)):
        pybullet.setJointMotorControl2(bodyIndex=robotID,
                                jointIndex=index,
                                controlMode=pybullet.POSITION_CONTROL,
                                targetPosition = config[index])
    time.sleep(0.02)

In [194]:
r2 = np.eye(3)
r1 = np.eye(3)
Config = geometric_ik([0.0,0.0,0.8-0.21],r1,[0.1,0.0,0.4],r2,0.155,0,0.135)
for index in range(pybullet.getNumJoints(robotID)):
    pybullet.setJointMotorControl2(bodyIndex=robotID,
                            jointIndex=index,
                            controlMode=pybullet.POSITION_CONTROL,
                            targetPosition = Config[index])

Analytic IK Prerequisities

In [58]:
def transformation(pos,attitude):
    alpha,beta,gamma = attitude
    x,y,z = pos
    rot = np.array([[np.cos(alpha)*np.cos(beta),np.cos(alpha)*np.sin(beta)*np.sin(gamma)-np.sin(alpha)*np.cos(gamma),np.cos(alpha)*np.sin(beta)*np.cos(gamma)+np.sin(alpha)*np.sin(gamma)],
                    [np.sin(alpha)*np.cos(beta),np.sin(alpha)*np.sin(beta)*np.sin(gamma)+np.cos(alpha)*np.cos(gamma),np.sin(alpha)*np.sin(beta)*np.cos(gamma)-np.cos(alpha)*np.sin(gamma)],
                    [-np.sin(beta),np.cos(beta)*np.sin(gamma),np.cos(beta)*np.cos(gamma)]])
    return np.array([[rot[0][0],rot[0][1],rot[0][2],x],
                    [rot[1][0],rot[1][1],rot[1][2],y],
                    [rot[2][0],rot[2][1],rot[2][2],z],
                    [0,0,0,1]])
print(transformation([0,0,0],[0,0,0]))

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [-0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


Analytic Solution

In [324]:
def calculateIK(pose,attitude):
    l3 = 0.155
    l4 = 0.135
    l5 = 0.1

    trans = np.linalg.inv(transformation(pose,attitude))
    # Define n',s',a',p'
    n = trans[0:3,0]
    s = trans[0:3,1]
    a = trans[0:3,2]
    p = trans[0:3,3]
    
    """print(trans)
    print(s)
    print(s)
    print(p)"""
    
    r1 = pybullet.getEulerFromQuaternion(pybullet.getLinkState(robotID,1)[1])
    x1 = -np.cos(r1[2])*np.sin(r1[1])*np.sin(r1[0])-np.sin(r1[2])*np.cos(r1[0])
    z1 = np.cos(r1[1])*np.sin(r1[0])
    r2 = pybullet.getEulerFromQuaternion(pybullet.getLinkState(robotID,2)[1])
    x2 = -np.cos(r2[2])*np.sin(r2[1])*np.sin(r2[0])-np.sin(r2[2])*np.cos(r2[0])
    r3 = pybullet.getEulerFromQuaternion(pybullet.getLinkState(robotID,3)[1])
    z3 = np.cos(r3[1])*np.sin(r3[0])
    x3 = -np.cos(r3[2])*np.sin(r3[1])*np.sin(r3[0])-np.sin(r3[2])*np.cos(r3[0])
    r4 = pybullet.getEulerFromQuaternion(pybullet.getLinkState(robotID,4)[1])
    x4 = -np.cos(r4[2])*np.sin(r4[1])*np.sin(r4[0])-np.sin(r4[2])*np.cos(r4[0])
    r5 = pybullet.getEulerFromQuaternion(pybullet.getLinkState(robotID,5)[1])
    x5 = -np.cos(r5[2])*np.sin(r5[1])*np.sin(r5[0])-np.sin(r5[2])*np.cos(r5[0])
    
    
    KNEE = np.sign(np.dot([0,0,z3],np.cross([x3,0,0],[x4,0,0])))
    ANKLE = np.sign(np.dot([0,0,z1],np.cross([x1,0,0],[x2,0,0])))
    HIP = np.sign(np.dot(p,[x5,0,0]))

    
    # Calculate Theta 4
    C4 = ((p[0] + l5)**2 + p[1]**2 + p[2]**2 - l3**2 - l4**2) / (2*l3*l4)
    S4 = KNEE * np.sqrt(abs(1 - C4**2))
    theta4 = np.arctan2(C4,S4)
    
    # Calculate Theta 5
    theta5 = np.arctan2( -p[2],ANKLE * np.sqrt((p[0]+l5)**2 + p[1]**2)) - np.arctan2(S4 * l3, C4 * l3 + l4)
    
    # Calculate Theta 6
    theta6 = np.arctan2(p[1],-p[0] - l5)
    if (np.cos(theta4 + theta5) * l3 + np.cos(theta5) * l4) < 0:
        theta6 += np.pi
    
    # Calculate Theta 2
    C2 = np.sin(theta6) * s[0] + np.cos(theta6) * s[1]
    S2 = HIP * np.sqrt(1 - C2**2)
    theta2 = np.arctan2(S2,C2)
    
    # Calculate Theta 1
    theta1 = np.arctan2(-np.sin(theta6) * s[0] - np.cos(theta6)*s[1],
                       -np.sin(theta6) * n[0] - np.cos(theta6)*n[1])
    
    # Calculate theta 3
    theta3 = np.arctan2(a[2], np.cos(theta6) * a[0] - np.sin(theta6) * a[1]) - theta4 - theta5
    
    if S2 < 0:
        theta1 += np.pi
        theta3 += np.pi
    
    return ([theta1,theta2,theta3,theta4,theta5,theta6])

print(pybullet.calculateInverseKinematics(robotID,5,
                                        [0.0,0.1,0.45],pybullet.getQuaternionFromEuler([0,0,np.pi/2])))

print("-------------------------------------------")
print(calculateIK([0.05,0.1,0.45],[0,0,np.pi/2]))

(3.839731376338733e-05, -0.0008178495522937044, -1.1473830428648575, 1.211377370602843, -0.07120384387897707, 0.0006847750267498607)
-------------------------------------------
[6.8035933285964025e-18, 1.5707963267948966, 0.0, 1.5707963267948966, -1.5707963267948966, -1.6814535479687922]


Analytic(analistic) IK (by me)

In [316]:
## Create Same trajectory
trajectory = list("")
for i in range(300):
    trajectory.append(np.array([0.0,-0.1+0.001*i,0.15]))
## Solve IK and set Joint angles
for i in range(300):
    
    config = calculateIK(trajectory[i],[0,0,np.pi/2])
    for index in range(pybullet.getNumJoints(robotID)):
        pybullet.setJointMotorControl2(bodyIndex=robotID,
                                jointIndex=index,
                                controlMode=pybullet.POSITION_CONTROL,
                                targetPosition = config[index])
    time.sleep(0.02)

Shutdown

In [81]:
pybullet.resetSimulation()

In [198]:
pybullet.disconnect()

In [289]:
pybullet.setJointMotorControl2(bodyIndex=robotID,
                            jointIndex=1,
                            controlMode=pybullet.POSITION_CONTROL,
                            targetPosition = 0)

In [ ]:
print(np.arctan2())